In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout, GRU
from keras.utils import get_custom_objects
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [ ]:
seed = 1234
np.random.seed(seed)
plt.style.use('ggplot')
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [ ]:
xau = pd.read_csv('./Data/xauusd_5-min_data.csv')
xau['Datetime'] = pd.to_datetime(xau['Date'])
xau.set_index('Datetime', inplace=True)

In [ ]:
xau_scaler = MinMaxScaler()
xau = xau_scaler.fit_transform(pd.DataFrame(xau[['Close']]))
xau_len = len(xau)
xau_train = xau[:int(xau_len * 0.8)]
xau_val = xau[int(xau_len * 0.8):int(xau_len * 0.9)]
xau_test = xau[int(xau_len * 0.9):]

In [ ]:
def create_sliding_windows(data, lag, step = 1):
    len_data = len(data)
    x = []
    y = []
    for i in range(lag, len_data - lag, step):
        x.append(data[i - lag:i, 0])
        y.append(data[i:i + lag, 0])
    return np.array(x), np.array(y)

In [ ]:
train_x, train_y = create_sliding_windows(xau_train, 289)
val_x, val_y = create_sliding_windows(xau_val, 289)
test_x, test_y = create_sliding_windows(xau_test, 289, 289)

In [ ]:
def get_model() -> Sequential:
    model = Sequential()
    model.add(GRU(units=145, return_sequences=True, input_shape=(289,1)))
    model.add(Dropout(0.2))
    model.add(GRU(units=73, return_sequences=True))
    model.add(Dropout(0.2))
    model.add(GRU(units=73, return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(units=289))
    model.compile(optimizer=Adam(learning_rate=0.001),loss='mean_squared_error')
    return model

def save_model(model: Sequential, filename: str):
    model.save(f"./model/{filename}.model")

def load_model(filename) -> Sequential:
    model = get_model()
    model.load_weights(f"./model/{filename}.model")
    return model

In [ ]:
model = get_model()
callbacks = [EarlyStopping(patience=10, restore_best_weights=True), ReduceLROnPlateau(factor=0.1, patience=10)]
model.fit(train_x, train_y, validation_data=(val_x,val_y), batch_size=8192, epochs=180, callbacks=callbacks)

In [ ]:
predict = model.predict(test_x)
predict_vec = xau_scaler.inverse_transform(predict)
pre_result = np.reshape(predict_vec, [predict_vec.shape[0] * predict_vec.shape[1]])
test_y = np.reshape(test_y, [test_y.shape[0] * test_y.shape[1]])
datacompare = pd.DataFrame()
datacompare['Predict Data'] = pre_result
datacompare['Real Data'] = test_y

# Calculatre value of Root Mean Square Error 
def rmse(datatest, datapred):
    return np.round(np.sqrt(np.mean((datapred - datatest) ** 2)), 4)
print('Result Root Mean Square Error Prediction Model :',rmse(test_y, pre_result))

def mape(datatest, datapred): 
    return np.round(np.mean(np.abs((datatest - datapred) / datatest) * 100), 4)
    
print('Result Mean Absolute Percentage Error Prediction Model : ', mape(test_y, pre_result), '%')
# Create graph data test and prediction result
plt.figure(num=None, figsize=(300, 40), dpi=80,facecolor='w', edgecolor='k')
plt.title('Graph Comparison Data Actual and Data Prediction')
plt.plot(datacompare['Predict Data'], color='red',label='Predict Data')
plt.plot(datacompare['Real Data'], color='blue',label='Real Data')
plt.xlabel('Day')
plt.ylabel('Price')
plt.legend()
plt.show()